In [ ]:
from google.colab import drive
drive.mount('/content/drive')
folder_path = '/content/drive/My Drive/EOD_2_SN/SN/'

Mounted at /content/drive


In [ ]:
!pip install piq torch torchvision pillow pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
!pip uninstall torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Found existing installation: torch 2.5.1+cu124
Uninstalling torch-2.5.1+cu124:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/bin/torchfrtrace
    /usr/local/bin/torchrun
    /usr/local/lib/python3.11/dist-packages/functorch/*
    /usr/local/lib/python3.11/dist-packages/torch-2.5.1+cu124.dist-info/*
    /usr/local/lib/python3.11/dist-packages/torch/*
    /usr/local/lib/python3.11/dist-packages/torchgen/*
Proceed (Y/n)? n
Found existing installation: torchvision 0.20.1+cu124
Uninstalling torchvision-0.20.1+cu124:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/torchvision-0.20.1+cu124.dist-info/*
    /usr/local/lib/python3.11/dist-packages/torchvision.libs/libcudart.41118559.so.12
    /usr/local/lib/python3.11/dist-packages/torchvision.libs/libjpeg.ceea7512.so.62
    /usr/local/lib/python3.11/dist-packages/torchvision.libs/libnvjpeg.02b6d700.so.12
    /usr/local/lib/python3.11/dist-packages/torchvision.libs/

In [6]:
import torch
import os
from PIL import Image
import torchvision.transforms as T
import piq
from glob import glob
import pandas as pd
import tifffile as tiff

def make_metadata_with_brisque_score_tiff(folder_path, mode):
    scores = []
    metadata_list = []
    files = glob(folder_path + '*.tif')

    keys_to_keep = ['image_name', 'ap_aperturesize', 'ap_wd', "sv_file_name", "ap_actualkv" ]

    for img_name in files:
        print(f"Processing: {img_name}")
        image_path = img_name
        image = Image.open(image_path).convert('RGB')

        # Get image dimensions
        width, height = image.size

        # Crop the first 691 rows while keeping full width
        cropped_image = image.crop((0, 0, width, min(691, height)))

        # Convert cropped image to a tensor
        to_tensor = T.ToTensor()
        img_tensor = to_tensor(cropped_image).unsqueeze(0)  # shape: [1, C, H, W]

        # Compute BRISQUE score
        score = piq.brisque(img_tensor, data_range=1.0)
        scores.append(score.item())

        # Extract TIFF metadata
        with tiff.TiffFile(image_path) as tif:
            cz_sem_metadata = None
            for page in tif.pages:
                for tag in page.tags.values():
                    if tag.name == 'CZ_SEM':
                        cz_sem_metadata = tag.value
                        break
                if cz_sem_metadata:
                    break

            parsed_metadata = {}
            if cz_sem_metadata and isinstance(cz_sem_metadata, dict):
                for k in keys_to_keep:
                    value = cz_sem_metadata.get(k, None)
                    if isinstance(value, tuple) and len(value) > 0:
                        parsed_metadata[k] = value[1]  # Extract only the numeric value
                    else:
                        parsed_metadata[k] = value
            else:
                parsed_metadata = {k: None for k in keys_to_keep}

            metadata_list.append(parsed_metadata)

    # Save results to CSV
    metadata_df = pd.DataFrame(metadata_list)
    df = pd.DataFrame({'File': files, 'Score': scores})
    result_df = pd.concat([df, metadata_df], axis=1)
    header = mode != 'a'  # Write header only if mode is not append
    result_df.to_csv(folder_path + 'brisque_scores_with_metadata10.csv', index=False, header=header, mode=mode)

# Example usage
make_metadata_with_brisque_score_tiff(folder_path, mode='w')


Processing: /content/drive/My Drive/EOD_2_SN/SN/SE2_20250324_10-11-60_30_00_K_X_001.tif
Processing: /content/drive/My Drive/EOD_2_SN/SN/SE2_20250324_10-3-68_30_00_K_X_005.tif
Processing: /content/drive/My Drive/EOD_2_SN/SN/SE2_20250324_10-15-53_30_00_K_X_003.tif
Processing: /content/drive/My Drive/EOD_2_SN/SN/SE2_20250324_10-8-63_30_00_K_X_008.tif
Processing: /content/drive/My Drive/EOD_2_SN/SN/SE2_20250324_10-12-59_30_00_K_X_002.tif
Processing: /content/drive/My Drive/EOD_2_SN/SN/SE2_20250324_10-16-56_30_00_K_X_004.tif
Processing: /content/drive/My Drive/EOD_2_SN/SN/SE2_20250324_10-7-64_30_00_K_X_007.tif
Processing: /content/drive/My Drive/EOD_2_SN/SN/SE2_20250324_10-4-67_30_00_K_X_006.tif
Processing: /content/drive/My Drive/EOD_2_SN/SN/SE2_20250321_20--2036_30_00_K_X_009.tif
Processing: /content/drive/My Drive/EOD_2_SN/SN/SE2_20250321_20--24-37_30_00_K_X_010.tif
Processing: /content/drive/My Drive/EOD_2_SN/SN/SE2_20250324_20-27-47_30_00_K_X_013.tif
Processing: /content/drive/My Drive